In [1]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/larger_clap_music")
processor = AutoProcessor.from_pretrained("laion/larger_clap_music")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [3]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [4]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


# Process audio

In [5]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [6]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [7]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [8]:
print(audio_embeddings.shape)
# Size of the 2^9 vector

torch.Size([59, 512])


# Process text

In [9]:
# Fixed value
# Find a better way of defining the problem computationally
# Discrete: Rating from 0-1
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]
# Percieved: what we would think the emotion should be of the sound (before hearing the sound)
discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
# After listening to the sound, what do we think now
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

#Continuous value (scale from 1-9)
# Continous: value from -1, 0, 1 (like a rating scale)
dimensional_tags = ["positive", "relaxed", "awake"]
dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [10]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# NOTE: currently using only dimensional_captions_induced
# Replace 'text=dimensional_captions_induced' with the other categories in new cells
tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

In [11]:
tag_inputs_disc_i = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
tag_embeds_disc_i = model.get_text_features(**tag_inputs_disc_i)

In [12]:
tag_inputs_dim_p = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_dim_p = model.get_text_features(**tag_inputs_dim_p)

In [13]:
tag_inputs_disc_p = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_disc_p = model.get_text_features(**tag_inputs_disc_p)

# Generate outputs

In [14]:
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 3])


In [15]:
sims_disc_i = torch.matmul(audio_embeddings, tag_embeds_disc_i.T)
print(sims_disc_i.shape)

torch.Size([59, 5])


In [16]:
sims_dim_p = torch.matmul(audio_embeddings, tag_embeds_dim_p.T)
print(sims_dim_p.shape)

torch.Size([59, 3])


In [17]:
sims_disc_p = torch.matmul(audio_embeddings, tag_embeds_disc_p.T)
print(sims_disc_p.shape)

torch.Size([59, 5])


## Load csv files and extract related columns

In [18]:
IDim_path = '/content/Exp1/Data/IDim/'
#IDim: Indiced dimensional
IDim_responses = []

all_dfs = []

for file in os.listdir(IDim_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59


In [19]:
PDim_path = '/content/Exp1/Data/PDim/'
#PDim: Percieved dimensional
PDim_responses = []

all_dfs = []

for file in os.listdir(PDim_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

PDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(PDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(PDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(PDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59


In [20]:
IDisc_path = '/content/Exp1/Data/IDisc/'
#IDisc: Induced discrete
IDisc_responses = []

all_dfs = []

for file in os.listdir(IDisc_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['happiness'])
        current_row_sadness.append(df.iloc[i]['sadness'])
        current_row_anger.append(df.iloc[i]['anger'])
        current_row_tenderness.append(df.iloc[i]['tenderness'])
        current_row_fear.append(df.iloc[i]['fear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

IDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(IDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(IDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(IDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(IDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(IDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector: 59
Length of fear_vector: 59


In [21]:
PDisc_path = '/content/Exp1/Data/PDisc/'
#PDisc: Percieved discrete
PDisc_responses = []

all_dfs = []

for file in os.listdir(PDisc_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['happiness'])
        current_row_sadness.append(df.iloc[i]['sadness'])
        current_row_anger.append(df.iloc[i]['anger'])
        current_row_tenderness.append(df.iloc[i]['tenderness'])
        current_row_fear.append(df.iloc[i]['fear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

PDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(PDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(PDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(PDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(PDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(PDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector: 59
Length of fear_vector: 59


# Evaluate

In [22]:
# Try and find diferent scaling methods to increase the pearson, MAE, MAPE correlations at the end
# Dimensional scaling
# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

Scaled sims shape: torch.Size([59, 3])
Scaled sims (first 5 rows):
tensor([[7.1647, 7.0362, 6.7362],
        [5.4812, 5.3205, 4.9816],
        [3.6491, 3.4734, 3.0879],
        [3.1602, 3.1239, 2.7138],
        [7.9248, 7.7499, 7.4512],
        [8.4446, 8.2234, 7.9175],
        [2.8712, 2.7444, 2.4006],
        [4.6985, 4.6478, 4.2677],
        [6.2722, 6.0907, 5.6706],
        [5.0022, 4.9392, 4.5388]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 3])
Human ratings tensor (first 5 rows):
tensor([[4.4872, 4.8798, 4.5185],
        [4.7682, 5.3057, 4.7414],
        [5.3105, 5.7312, 5.0760],
        [5.2582, 5.2651, 5.4488],
        [3.8897, 3.9540, 5.3549],
        [4.3197, 3.8900, 5.5089],
        [4.9825, 5.1875, 5.5354],
        [5.0934, 4.8657, 5.8208],
        [4.7605, 4.8502, 5.4158],
        [5.2577, 4.8688, 5.7003]])



In [23]:
# Find the min and max values in the current sims tensor
old_min_disc_i = sims_disc_i.min()
old_max_disc_i = sims_disc_i.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_disc_i = ((sims_disc_i - old_min_disc_i) / (old_max_disc_i - old_min_disc_i)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_disc_i.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_i[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_i.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_i.max():.4f}\n")

human_ratings_tensor_disc_i = torch.tensor([
    IDisc_responses['happiness_vector'],
    IDisc_responses['sadness_vector'],
    IDisc_responses['anger_vector'],
    IDisc_responses['tenderness_vector'],
    IDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_i.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_i[:10]}\n")

Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[6.6468, 7.1930, 7.2343, 7.1569, 6.6206],
        [4.9902, 5.4831, 5.5595, 5.5152, 4.9526],
        [3.1643, 3.6584, 3.7633, 3.6901, 3.1061],
        [2.7430, 3.2034, 3.2678, 3.1855, 2.7183],
        [7.3257, 7.8584, 7.9318, 7.8513, 7.3208],
        [7.8057, 8.3000, 8.4567, 8.3852, 7.8269],
        [2.3419, 2.8381, 3.0018, 2.9091, 2.3334],
        [4.2095, 4.6734, 4.7742, 4.6939, 4.1943],
        [5.7331, 6.2517, 6.3309, 6.2626, 5.6885],
        [4.5250, 4.9970, 5.0665, 4.9898, 4.4868]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.8202, 3.3418, 2.7948, 1.9789, 4.1430],
        [1.9091, 3.4459, 2.9059, 2.3785, 2.8558],
        [2.0644, 3.1847, 2.4182, 2.6371, 2.3356],
        [2.5321, 3.1900, 2.1509, 3.0583, 2.0938],
        [1.8391, 2.4733, 4.0144, 1.9902, 3.4942],
        [

In [24]:
# Find the min and max values in the current sims tensor
old_min_disc_p = sims_disc_p.min()
old_max_disc_p = sims_disc_p.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_disc_p = ((sims_disc_p - old_min_disc_p) / (old_max_disc_p - old_min_disc_p)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_disc_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_p.max():.4f}\n")

human_ratings_tensor_disc_p = torch.tensor([
    PDisc_responses['happiness_vector'],
    PDisc_responses['sadness_vector'],
    PDisc_responses['anger_vector'],
    PDisc_responses['tenderness_vector'],
    PDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_p[:10]}\n")

Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[6.6321, 7.2144, 7.2363, 7.2299, 6.6378],
        [5.0138, 5.5393, 5.5996, 5.6218, 5.0076],
        [3.2399, 3.7695, 3.8560, 3.8417, 3.2128],
        [2.7646, 3.2602, 3.3098, 3.2947, 2.7788],
        [7.2153, 7.7868, 7.8401, 7.8417, 7.2457],
        [7.6816, 8.2146, 8.3517, 8.3653, 7.7390],
        [2.2927, 2.8278, 2.9737, 2.9531, 2.3213],
        [4.1486, 4.6512, 4.7398, 4.7369, 4.1799],
        [5.7722, 6.3281, 6.3864, 6.3824, 5.7587],
        [4.4855, 4.9941, 5.0510, 5.0487, 4.4900]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.5362, 4.4191, 5.1551, 2.0345, 5.6468],
        [1.7403, 4.8642, 4.2637, 2.5285, 3.9835],
        [2.0486, 5.5669, 3.0558, 2.9492, 2.8565],
        [2.4389, 4.8343, 2.1514, 3.5546, 3.0082],
        [1.5691, 3.6631, 5.0478, 1.9322, 4.7326],
        [

In [25]:
# Find the min and max values in the current sims tensor
old_min_dim_p = sims_dim_p.min()
old_max_dim_p = sims_dim_p.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_dim_p = ((sims_dim_p - old_min_dim_p) / (old_max_dim_p - old_min_dim_p)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_dim_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_dim_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_dim_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_dim_p.max():.4f}\n")

human_ratings_tensor_dim_p = torch.tensor([
    PDim_responses['positive_vector'],
    PDim_responses['relaxed_vector'],
    PDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor_dim_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_dim_p[:10]}\n")

Scaled sims shape: torch.Size([59, 3])
Scaled sims (first 5 rows):
tensor([[7.2057, 7.0539, 6.7190],
        [5.5631, 5.3781, 5.0007],
        [3.7865, 3.5891, 3.1684],
        [3.2481, 3.1847, 2.7345],
        [7.8737, 7.6719, 7.3367],
        [8.3794, 8.1321, 7.7885],
        [2.8932, 2.7385, 2.3604],
        [4.7022, 4.6258, 4.2052],
        [6.3784, 6.1654, 5.7069],
        [5.0270, 4.9361, 4.4959]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 3])
Human ratings tensor (first 5 rows):
tensor([[3.5116, 4.4973, 3.4563],
        [4.1519, 4.9952, 3.2785],
        [4.6982, 5.4154, 4.2416],
        [5.2018, 5.2612, 4.9213],
        [3.1964, 3.4387, 3.9169],
        [3.9328, 3.8416, 4.5261],
        [4.5933, 4.8528, 4.9660],
        [4.5391, 4.6797, 5.4388],
        [4.2891, 4.7658, 4.2599],
        [4.6275, 4.4824, 5.1707]])



 # Comparison Metrics

In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr # For Pearson correlation

# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
mae_disc_i = torch.mean(torch.abs(scaled_sims_disc_i - human_ratings_tensor_disc_i))
mae_disc_p = torch.mean(torch.abs(scaled_sims_disc_p - human_ratings_tensor_disc_p))
mae_dim_p = torch.mean(torch.abs(scaled_sims_dim_p - human_ratings_tensor_dim_p))

print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : {mae:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : {mae_disc_i:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : {mae_disc_p:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : {mae_dim_p:.4f}")

# Comparison Method 2: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).

#IDim
correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())

#PDim
correlation_positive_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 0].detach().numpy(), human_ratings_tensor_dim_p[:, 0].numpy())
correlation_relaxed_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 1].detach().numpy(), human_ratings_tensor_dim_p[:, 1].numpy())
correlation_awake_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 2].detach().numpy(), human_ratings_tensor_dim_p[:, 2].numpy())

# IDisc
correlation_happiness, _ = pearsonr(scaled_sims_disc_i[:, 0].detach().numpy(), human_ratings_tensor_disc_i[:, 0].numpy())
correlation_sadness, _ = pearsonr(scaled_sims_disc_i[:, 1].detach().numpy(), human_ratings_tensor_disc_i[:, 1].numpy())
correlation_anger, _ = pearsonr(scaled_sims_disc_i[:, 2].detach().numpy(), human_ratings_tensor_disc_i[:, 2].numpy())
correlation_tenderness, _ = pearsonr(scaled_sims_disc_i[:, 3].detach().numpy(), human_ratings_tensor_disc_i[:, 3].numpy())
correlation_fear, _ = pearsonr(scaled_sims_disc_i[:, 4].detach().numpy(), human_ratings_tensor_disc_i[:, 4].numpy())

#PDisc
correlation_happiness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 0].detach().numpy(), human_ratings_tensor_disc_p[:, 0].numpy())
correlation_sadness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 1].detach().numpy(), human_ratings_tensor_disc_p[:, 1].numpy())
correlation_anger_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 2].detach().numpy(), human_ratings_tensor_disc_p[:, 2].numpy())
correlation_tenderness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 3].detach().numpy(), human_ratings_tensor_disc_p[:, 3].numpy())
correlation_fear_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 4].detach().numpy(), human_ratings_tensor_disc_p[:, 4].numpy())

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:")
print(f"  Positive Dimension: {correlation_positive_dim_p:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed_dim_p:.4f}")
print(f"  Awake Dimension:    {correlation_awake_dim_p:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:")
print(f"  Happiness Dimension: {correlation_happiness:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness:.4f}")
print(f"  Anger Dimension:    {correlation_anger:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness:.4f}")
print(f"  Fear Dimension:    {correlation_fear:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:")
print(f"  Happiness Dimension: {correlation_happiness_disc_p:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness_disc_p:.4f}")
print(f"  Anger Dimension:    {correlation_anger_disc_p:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness_disc_p:.4f}")
print(f"  Fear Dimension:    {correlation_fear_disc_p:.4f}\n")



average_correlation = (correlation_positive + correlation_relaxed + correlation_awake) / 3
average_correlation_disc_i = (correlation_happiness + correlation_sadness + correlation_anger + correlation_tenderness + correlation_fear) / 5
average_correlation_disc_p = (correlation_happiness_disc_p + correlation_sadness_disc_p + correlation_anger_disc_p + correlation_tenderness_disc_p + correlation_fear_disc_p) / 5
average_correlation_dim_p = (correlation_positive_dim_p + correlation_relaxed_dim_p + correlation_awake_dim_p) / 3

print(f"  Average Correlation (IDim): {average_correlation:.4f}")
print(f"  Average Correlation (IDisc): {average_correlation_disc_i:.4f}")
print(f"  Average Correlation (PDisc): {average_correlation_disc_p:.4f}")
print(f"  Average Correlation (PDim): {average_correlation_dim_p:.4f}")

Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : 1.8040
Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : 3.3042
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : 2.9056
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : 1.8371
Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:
  Positive Dimension: 0.4015
  Relaxed Dimension:  0.3448
  Awake Dimension:    0.0842

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:
  Positive Dimension: 0.3893
  Relaxed Dimension:  0.3634
  Awake Dimension:    0.0251

Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:
  Happiness Dimension: 0.4549
  Sadness Dimension:  -0.4340
  Anger Dimension:    -0.3218
  Tenderness Dimension:    0.3611
  Fear Dimension:    -0.1315

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:
  Happiness Dimension: 0.3447


In [28]:
# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
absolute_percentage_error_disc_i = torch.abs((human_ratings_tensor_disc_i - scaled_sims_disc_i) / human_ratings_tensor_disc_i) * 100
absolute_percentage_error_disc_p = torch.abs((human_ratings_tensor_disc_p - scaled_sims_disc_p) / human_ratings_tensor_disc_p) * 100
absolute_percentage_error_dim_p = torch.abs((human_ratings_tensor_dim_p - scaled_sims_dim_p) / human_ratings_tensor_dim_p) * 100



# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)
mape_disc_i = torch.mean(absolute_percentage_error_disc_i)
mape_disc_p = torch.mean(absolute_percentage_error_disc_p)
mape_dim_p = torch.mean(absolute_percentage_error_dim_p)

print(f"Mean Absolute Percentage Error (MAPE) IDim: {mape:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) IDisc: {mape_disc_i:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDisc: {mape_disc_p:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDim: {mape_dim_p:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2
squared_differences_disc_i = (human_ratings_tensor_disc_i - scaled_sims_disc_i)**2
squared_differences_disc_p = (human_ratings_tensor_disc_p - scaled_sims_disc_p)**2
squared_differences_dim_p = (human_ratings_tensor_dim_p - scaled_sims_dim_p)**2


# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)
mse_disc_i = torch.mean(squared_differences_disc_i)
mse_disc_p = torch.mean(squared_differences_disc_p)
mse_dim_p = torch.mean(squared_differences_dim_p)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)
rmse_disc_i = torch.sqrt(mse_disc_i)
rmse_disc_p = torch.sqrt(mse_disc_p)
rmse_dim_p = torch.sqrt(mse_dim_p)

print(f"Root Mean Squared Error (RMSE) IDim: {rmse:.4f}\n")
print(f"Root Mean Squared Error (RMSE) IDisc: {rmse_disc_i:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDisc: {rmse_disc_p:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDim: {rmse_dim_p:.4f}\n")

from sklearn.metrics import r2_score

# R-squared IDim
print("\nR-squared scores (IDim):")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print("  valence =", r2_valence)
r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 0].detach().numpy())
print("  tension =", r2_tension)
r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 0].detach().numpy())
print("  energy =", r2_energy)

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_valence_disc_i = r2_score(human_ratings_tensor_disc_i[:, 0].numpy(), scaled_sims_disc_i[:, 0].detach().numpy())
print("  valence =", r2_valence_disc_i)
r2_tension_disc_i = r2_score(human_ratings_tensor_disc_i[:, 1].numpy(), scaled_sims_disc_i[:, 0].detach().numpy())
print("  tension =", r2_tension_disc_i)
r2_energy_disc_i = r2_score(human_ratings_tensor_disc_i[:, 2].numpy(), scaled_sims_disc_i[:, 0].detach().numpy())
print("  energy =", r2_energy_disc_i)

# R-squared PDim
print("\nR-squared scores (PDim):")
r2_valence_dim_p = r2_score(human_ratings_tensor_dim_p[:, 0].numpy(), scaled_sims_dim_p[:, 0].detach().numpy())
print("  valence =", r2_valence_dim_p)
r2_tension_dim_p = r2_score(human_ratings_tensor_dim_p[:, 1].numpy(), scaled_sims_dim_p[:, 0].detach().numpy())
print("  tension =", r2_tension_dim_p)
r2_energy_dim_p = r2_score(human_ratings_tensor_dim_p[:, 2].numpy(), scaled_sims_dim_p[:, 0].detach().numpy())
print("  energy =", r2_energy_dim_p)

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_valence_disc_p = r2_score(human_ratings_tensor_disc_p[:, 0].numpy(), scaled_sims_disc_p[:, 0].detach().numpy())
print("  valence =", r2_valence_disc_p)
r2_tension_disc_p = r2_score(human_ratings_tensor_disc_p[:, 1].numpy(), scaled_sims_disc_p[:, 0].detach().numpy())
print("  tension =", r2_tension_disc_p)
r2_energy_disc_p = r2_score(human_ratings_tensor_disc_p[:, 2].numpy(), scaled_sims_disc_p[:, 0].detach().numpy())
print("  energy =", r2_energy_disc_p)


Mean Absolute Percentage Error (MAPE) IDim: 39.74%
Mean Absolute Percentage Error (MAPE) IDisc: 148.60%
Mean Absolute Percentage Error (MAPE) PDisc: 115.60%
Mean Absolute Percentage Error (MAPE) PDim: 40.89%

Root Mean Squared Error (RMSE) IDim: 2.1785

Root Mean Squared Error (RMSE) IDisc: 3.8723

Root Mean Squared Error (RMSE) PDisc: 3.4905

Root Mean Squared Error (RMSE) PDim: 2.2359


R-squared scores (IDim):
  valence = -2.7064833641052246
  tension = -2.5840413570404053
  energy = -11.137897491455078

R-squared scores (IDisc):
  valence = -12.6892671585083
  tension = -36.159339904785156
  energy = -10.70843505859375

R-squared scores (PDim):
  valence = -2.350156784057617
  tension = -2.7900383472442627
  energy = -4.042289733886719

R-squared scores (PDisc):
  valence = -7.210134506225586
  tension = -8.353472709655762
  energy = -7.405060768127441
